<a href="https://colab.research.google.com/github/kokbent/covid-safegraph/blob/master/Colab/combine_safegraph_sdm_FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import re
import json
import math
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
destination = '/content/gdrive/My Drive/safegraph-sdm'

In [4]:
def getDate(f):
  split = re.split('\.|-', f)[3:6]
  date = '-'.join(split)
  return(date)

def calcOutstate(js):
  dic = js.items()
  instate = outstate = 0
  for (k, v) in dic:
    if k.startswith('12'):
      instate += v
    else:
      outstate += v
  return(outstate/(instate + outstate))

def calcEntropy(js):
  tot = sum(js.values())
  tmp1 = list(js.values())
  p = [i/tot for i in tmp1]
  entropy = sum([- i * math.log(i) for i in p])
  return(entropy)

def calcDestNum(js):
  l = len(js.values())
  return(l)

In [ ]:
#df = pd.read_csv(destination + '/' + 'fl-social-distancing-2019-01-01.csv')
#df

,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home,mean_home_dwell_time,mean_non_home_dwell_time,mean_distance_traveled_from_home,origin_census_block_group1
0,120050014032,2019-01-01T00:00:00-06:00,2019-01-02T00:00:00-06:00,23,3247,"{""""16001-50000"""":2,""""0"""":5,"""">50000"""":4,""""2001...","{""""16001-50000"""":198,"""">50000"""":20,""""<1000"""":4...",10,617,"{""""721-1080"""":4,""""361-720"""":1,""""61-360"""":3,""""<...","[9,12,10,10,10,15,10,10,9,8,8,9,2,5,6,7,9,9,10...",3,1,"{""""120050016001"""":2,""""120910233081"""":1,""""12005...",1,80,83,"{""""21-45"""":1,""""481-540"""":1,""""301-360"""":1,""""<20...",90,"{""""0-25"""":6,""""76-100"""":12,""""51-75"""":3}",605,296,67775,120050014032
1,120310173002,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,93,2719,"{""""16001-50000"""":11,""""0"""":38,"""">50000"""":18,""""2...","{""""16001-50000"""":39,"""">50000"""":53,""""<1000"""":16...",34,659,"{""""721-1080"""":16,""""361-720"""":8,""""61-360"""":8,""""...","[46,47,46,46,47,45,48,47,46,42,44,39,36,38,37,...",1,2,"{""""516600001022"""":1,""""081139681011"""":1,""""12031...",1,23,225,"{""""21-45"""":1,""""481-540"""":1,""""541-600"""":4,""""46-...",92,"{""""0-25"""":28,""""76-100"""":54,""""51-75"""":9,""""26-50...",646,231,21178,120310173002
2,120710403093,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,235,5683,"{""""16001-50000"""":40,""""0"""":107,"""">50000"""":20,""""...","{""""16001-50000"""":89,"""">50000"""":59,""""<1000"""":24...",107,944,"{""""721-1080"""":37,""""361-720"""":26,""""61-360"""":23,...","[133,138,143,147,145,145,148,146,143,140,141,1...",6,1,"{""""120710504003"""":4,""""470370195004"""":1,""""12071...",6,8,668,"{""""21-45"""":8,""""481-540"""":2,""""46-60"""":2,""""721-8...",99,"{""""0-25"""":33,""""76-100"""":167,""""51-75"""":20,""""26-...",822,189,20001,120710403093
3,120830025022,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,443,1795,"{""""16001-50000"""":20,""""0"""":192,"""">50000"""":69,""""...","{""""16001-50000"""":101,"""">50000"""":113,""""<1000"""":...",191,720,"{""""721-1080"""":66,""""361-720"""":51,""""61-360"""":59,...","[208,225,225,226,229,229,226,226,222,220,212,2...",13,5,"{""""280790405001"""":1,""""130570910013"""":2,""""12069...",4,12,1319,"{""""21-45"""":24,""""481-540"""":5,""""541-600"""":4,""""46...",97,"{""""0-25"""":100,""""76-100"""":285,""""51-75"""":24,""""26...",690,193,17986,120830025022
4,120860039181,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,49,1536,"{""""16001-50000"""":2,""""0"""":18,"""">50000"""":4,""""200...","{""""16001-50000"""":54,"""">50000"""":360,""""<1000"""":8...",16,702,"{""""721-1080"""":5,""""361-720"""":6,""""61-360"""":9,""""<...","[16,17,20,21,23,22,25,24,24,26,24,23,26,22,31,...",1,3,"{""""120860037031"""":1,""""120860001192"""":1,""""12086...",1,63,151,"{""""21-45"""":2,""""481-540"""":1,""""721-840"""":1,""""120...",92,"{""""0-25"""":10,""""76-100"""":23,""""51-75"""":4,""""26-50...",708,286,3007,120860039181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11368,120310155023,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,167,1725,"{""""16001-50000"""":9,""""0"""":79,"""">50000"""":11,""""20...","{""""16001-50000"""":89,"""">50000"""":130,""""<1000"""":8...",80,446,"{""""721-1080"""":24,""""361-720"""":27,""""61-360"""":35,...","[74,76,79,85,86,86,87,78,79,78,73,75,76,73,72,...",5,2,"{""""120310123003"""":2,""""120310143382"""":1,""""12031...",2,5,532,"{""""21-45"""":5,""""481-540"""":1,""""46-60"""":3,""

In [5]:
files = os.listdir(destination)
df = []

for f in tqdm(files):
  if f.endswith('.csv'):
    dt = getDate(f)
    dat = pd.read_csv(destination + '/' + f)
    dat.bucketed_distance_traveled = dat.bucketed_distance_traveled.str.replace('""', '"')
    dat.destination_cbgs = dat.destination_cbgs.str.replace('""', '"')

    dat1 = dat[['origin_census_block_group', 'median_home_dwell_time', 'median_percentage_time_home', 'device_count', 'median_non_home_dwell_time', 'distance_traveled_from_home']]
    dat1.insert(0, 'date', dt)

    #dat2 = pd.json_normalize(dat.bucketed_distance_traveled.apply(json.loads))
      
    #tmp = dat.destination_cbgs.apply(json.loads)
    #dat3 = tmp.apply(calcOutstate)
    #dat3 = dat3.rename('outstate_prop')

    #dat4 = tmp.apply(calcEntropy)
    #dat4 = dat4.rename('destination_entropy')

    #dat5 = tmp.apply(calcDestNum)
    #dat5 = dat5.rename('number_of_destination')

    #df.append(pd.concat([dat1, dat2, dat3, dat4, dat5], axis = 1))
    df.append(dat1)

bigdf = pd.concat(df)

100%|██████████| 599/599 [11:40<00:00,  1.17s/it]


In [6]:
bigdf.to_csv('/content/gdrive/My Drive/sdm-consolidated-2020-08-21-reduced.csv', index=False)

In [7]:
bigdf.shape

(6763833, 7)

# Scrap

In [ ]:
files = os.listdir(destination)
f = files[0]
df = []

if f.endswith('.csv'):
  dt = getDate(f)
  dat = pd.read_csv(destination + '/' + f)
  dat.bucketed_distance_traveled = dat.bucketed_distance_traveled.str.replace('""', '"')
  dat.destination_cbgs = dat.destination_cbgs.str.replace('""', '"')

  dat1 = dat[['origin_census_block_group', 'median_home_dwell_time', 'median_percentage_time_home', 'device_count']]
  dat1.insert(0, 'date', dt)

  dat2 = pd.json_normalize(dat.bucketed_distance_traveled.apply(json.loads))
    
  tmp = dat.destination_cbgs.apply(json.loads)
  dat3 = tmp.apply(calcOutstate)
  dat3 = dat3.rename('outstate_prop')

  dat4 = tmp.apply(calcEntropy)
  dat4 = dat4.rename('destination_entropy')

  dat5 = tmp.apply(calcDestNum)
  dat5 = dat5.rename('number_of_destination')

  df.append(pd.concat([dat1, dat2, dat3, dat4, dat5], axis = 1))

df[0]